# Grant Perkins CS 4513 Final Project

In this project, I developed a distributed machine learning solution with AWS SageMaker.

Features:
 - Distributed computing:
   + One virtual computer runs this Jupyter Notebook
   + One virtual computer stores dataset
   + N virtual computers run the training job
   + One virtual computer runs the inference job (small load)
 - Machine Learning:
   + Trains a custom neural network to recognize digits in the MNIST dataset
   + Customizable hyperparameters (epochs, batch size, learning rate)
   + Inference on whatever image (classification)

I started this project on 4/30/2021, for CS 4513.

## Section 1: Data

In this section, I download the dataset, and upload it to a computer controlled by AWS S3, a long-term data service.

In [3]:
N_workers = 16  # must be factor of 8
epochs = 100
batch_size = 128  # one batch per epoch
learning_rate = 0.000125

hyperparameters = {
    "epochs": epochs,
    "batch_size": batch_size,
    "learning_rate": learning_rate
}

In [5]:
import tensorflow as tf
import sagemaker as sage
from sagemaker.tensorflow import TensorFlow

In [6]:
# folder within S3 bucket
bucket = "CS4513MNIST"
# temp download location of MNIST dataset
mnist_directory = "/home/ec2-user/.keras/datasets"
sess = sage.Session()
role = sage.get_execution_role()

In [7]:
for i in range(N_workers):
    tf.keras.datasets.mnist.load_data(path='mnist-%d.npz' % i)
data_location = sess.upload_data(mnist_directory, key_prefix=bucket)

## Section 2: Training

Now that the data is in the correct place, I can train a neural network for the dataset. I do this on N separate computers. N can be any natural number.

In [8]:
estimator = TensorFlow(
    base_job_name='mnist-cs4513',
    source_dir='code',
    entry_point='mnist.py',
    role=role,
    py_version='py37',
    framework_version='2.4.1',
    hyperparameters=hyperparameters,
    instance_count=N_workers // 8,
    instance_type='ml.p3.16xlarge',
    sagemaker_session=sess,
    distribution={
        'smdistributed': {
            'dataparallel': {
                'enabled': True
            }
        }
    }
)

estimator.fit(data_location)

2021-05-01 00:47:42 Starting - Starting the training job...
2021-05-01 00:47:54 Starting - Launching requested ML instancesProfilerReport-1619830061: InProgress
.........
2021-05-01 00:49:36 Starting - Preparing the instances for training............
2021-05-01 00:51:38 Downloading - Downloading input data......
2021-05-01 00:52:37 Training - Downloading the training image......
2021-05-01 00:53:37 Training - Training image download completed. Training in progress.2021-05-01 00:53:37.095822: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2021-05-01 00:53:37.103772: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2021-05-01 00:53:37.261683: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-05-01 00:53:37.403226: W tensorflow/

UnexpectedStatusException: Error for Training job mnist-cs4513-2021-05-01-00-47-41-879: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
Command "mpirun --host algo-1:8,algo-2:8 -np 16 --allow-run-as-root --tag-output --oversubscribe -mca btl_tcp_if_include eth0 -mca oob_tcp_if_include eth0 -mca plm_rsh_no_tree_spawn 1 -mca pml ob1 -mca btl ^openib -mca orte_abort_on_non_zero_status 1 -mca btl_vader_single_copy_mechanism none -mca plm_rsh_num_concurrent 2 -x NCCL_SOCKET_IFNAME=eth0 -x NCCL_DEBUG=INFO -x LD_LIBRARY_PATH -x PATH -x SMDATAPARALLEL_USE_HOMOGENEOUS=1 -x FI_PROVIDER=efa -x RDMAV_FORK_SAFE=1 -x LD_PRELOAD=/usr/local/lib/python3.7/site-packages/gethostname.cpython-37m-x86_64-linux-gnu.so -x SMDATAPARALLEL_SERVER_ADDR=algo-1 -x SMDATAPARALLEL_SERVER_PORT=7592 -x SAGEMAKER_INSTANCE_TYPE=ml.p3.16xlarge smddprun /usr/local/bin/python3.7 -m mpi4py mnist.py --N_workers 16 --batch_size 128 --epochs 100 --learning_rate 0.000125 --model_dir s3://sagemaker-us-east-1-118451457254/